
# ⏳ Prévision Temporelle des Coûts (Time Series)

## Objectif
Ce notebook a pour but d'entraîner un modèle spécifique capable de prédire les coûts de traitement des déchets pour une **date future donnée**.

Contrairement au modèle classique qui se base sur les caractéristiques du déchet (type, poids, etc.), ce modèle se concentre sur l'**historique temporel** et les **tendances** par hôpital.

### Étapes :
1.  Chargement des données.
2.  **Feature Engineering Temporel** : Création de variables explicatives à partir de la date (année, mois, jour, jour de la semaine...).
3.  **Encodage** : Transformation des noms d'hôpitaux en valeurs numériques.
4.  **Entraînement** : Utilisation d'un RandomForestRegressor.
5.  **Sauvegarde** : Export du modèle pour l'application Web.


In [ ]:

import pandas as pd
import numpy as np
import joblib
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Configuration pour l'affichage
sns.set(style="whitegrid")
%matplotlib inline

# Définition des chemins
BASE_DIR = "../../web_app"
DATA_PATH = "../../notebooks/data/dechets_hospitaliers.csv"
MODELS_DIR = os.path.join(BASE_DIR, "models")

# Création du dossier models s'il n'existe pas
os.makedirs(MODELS_DIR, exist_ok=True)


## 1. Chargement des Données

In [ ]:

print("Chargement du dataset...")
df = pd.read_csv(DATA_PATH)

# Conversion de la colonne date
df['date_collecte'] = pd.to_datetime(df['date_collecte'])

display(df.head())
df.info()



## 2. Feature Engineering Temporel

Pour qu'un modèle puisse comprendre la notion de "temps" et de "saisonnalité", nous devons décomposer la date en plusieurs caractéristiques numériques :
*   **Année** : Pour capter la tendance globale (inflation, augmentation du volume...).
*   **Mois** : Pour la saisonnalité annuelle.
*   **Jour du mois** : Pour les cycles mensuels.
*   **Jour de la semaine** : Pour les cycles hebdomadaires (ex: moins de collectes le week-end).
*   **Jour de l'année** : Pour une granularité fine.


In [ ]:

df['annee'] = df['date_collecte'].dt.year
df['mois'] = df['date_collecte'].dt.month
df['jour'] = df['date_collecte'].dt.day
df['jour_semaine'] = df['date_collecte'].dt.dayofweek
df['jour_annee'] = df['date_collecte'].dt.dayofyear

print("Aperçu des nouvelles features temporelles :")
display(df[['date_collecte', 'annee', 'mois', 'jour', 'jour_semaine']].head())



## 3. Encodage des Hôpitaux

Le modèle a besoin de savoir pour quel hôpital il prédit. Comme les algorithmes ne comprennent que les nombres, nous utilisons un **LabelEncoder** pour transformer chaque nom d'hôpital en un nombre unique entier.

Il est CRUCIAL de sauvegarder cet encodeur pour que l'application Web puisse transformer les choix de l'utilisateur de la même manière.


In [ ]:

le_hopital = LabelEncoder()
df['hopital_encoded'] = le_hopital.fit_transform(df['hopital'])

# Vérification
print(f"Nombre d'hôpitaux uniques : {len(le_hopital.classes_)}")
display(df[['hopital', 'hopital_encoded']].head())


## 4. Visualisation de l'Évolution Temporelle

In [ ]:

# Agrégation par mois pour voir la tendance globale
df['mois_annee'] = df['date_collecte'].dt.to_period('M')
monthly_cost = df.groupby('mois_annee')['cout_traitement'].sum()

plt.figure(figsize=(12, 6))
monthly_cost.plot(kind='line', marker='o', color='teal')
plt.title('Évolution du Coût Total Mensuel')
plt.ylabel('Coût ($)')
plt.xlabel('Mois')
plt.grid(True)
plt.show()



## 5. Entraînement du Modèle

Nous utilisons un **RandomForestRegressor**. C'est un modèle robuste capable de capturer des relations non-linéaires complexes entre la date, l'hôpital et le coût.


In [ ]:

# Définition des features (X) et de la cible (y)
features_cols = ['hopital_encoded', 'annee', 'mois', 'jour', 'jour_semaine', 'jour_annee']
X = df[features_cols]
y = df['cout_traitement']

print("Entraînement du modèle de COÛT en cours...")
model_cout = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_cout.fit(X, y)
print(f"✅ Modèle COÛT (R²) : {model_cout.score(X, y):.4f}")

# 2. Modèle QUANTITÉ (Regression)
print("Entraînement du modèle de QUANTITÉ en cours...")
y_qty = df['poids_kg']
model_qty = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_qty.fit(X, y_qty)
print(f"✅ Modèle QUANTITÉ (R²) : {model_qty.score(X, y_qty):.4f}")

# 3. Modèle TYPE (Classification)
print("Entraînement du modèle de TYPE en cours...")
le_type = LabelEncoder()
y_type = le_type.fit_transform(df['type_dechet'])

model_type = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model_type.fit(X, y_type)
print(f"✅ Modèle TYPE (Accuracy) : {model_type.score(X, y_type):.4f}")

# 4. Modèle RISQUE (Classification)
print("Entraînement du modèle de RISQUE en cours...")
# Gestion des NaN pour le risque
df['niveau_risque'] = df['niveau_risque'].fillna('Moyen')
le_risk = LabelEncoder()
y_risk = le_risk.fit_transform(df['niveau_risque'])

model_risk = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model_risk.fit(X, y_risk)
print(f"✅ Modèle RISQUE (Accuracy) : {model_risk.score(X, y_risk):.4f}")



## 6. Sauvegarde des Artefacts

Nous sauvegardons :
1.  Le modèle entraîné (`model_future_cout.pkl`).
2.  L'encodeur des hôpitaux (`encoder_hopital_future.pkl`).

Ces fichiers seront chargés par l'application Flask via le fichier `future_routes.py`.


In [ ]:

# Sauvegarde COÛT
joblib.dump(model_cout, os.path.join(MODELS_DIR, 'model_future_cout.pkl'))
joblib.dump(le_hopital, os.path.join(MODELS_DIR, 'encoder_hopital_future.pkl'))

# Sauvegarde QUANTITÉ
joblib.dump(model_qty, os.path.join(MODELS_DIR, 'model_future_qty.pkl'))

# Sauvegarde TYPE
joblib.dump(model_type, os.path.join(MODELS_DIR, 'model_future_type.pkl'))
joblib.dump(le_type, os.path.join(MODELS_DIR, 'encoder_future_type.pkl'))

# Sauvegarde RISQUE
joblib.dump(model_risk, os.path.join(MODELS_DIR, 'model_future_risk.pkl'))
joblib.dump(le_risk, os.path.join(MODELS_DIR, 'encoder_future_risk.pkl'))

print(f"✅ Tous les modèles et encodeurs ont été sauvegardés dans : {MODELS_DIR}")
